# OpenMDAO Example

_Just to ensure profiler is working and show how to use it_

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import openmdao.api as om
import ipymodelview 

### Sellar Problem

We will reuse the [Sellar problem](https://openmdao.org/newdocs/versions/latest/basic_user_guide/multidisciplinary_optimization/sellar.html) from the OpenMDAO documention. It is an example of two-discipline optimization problem with feedback coupling. 

#### Disciplines

In [3]:
class SellarDis1(om.ExplicitComponent):
    """
    Component containing Discipline 1 -- no derivatives version.
    """
    
    def initialize(self):
        self.options.declare('num_nodes', types=int, default=100)

    def setup(self):
        nn = self.options["num_nodes"]

        # Global Design Variable
        self.add_input('z1', val=np.zeros(nn,))
        self.add_input('z2', val=np.zeros(nn,))

        # Local Design Variable
        self.add_input('x', val=np.zeros(nn,))

        # Coupling parameter
        self.add_input('y2', val=np.ones(nn,))

        # Coupling output
        self.add_output('y1', val=np.ones(nn,))

    def setup_partials(self):
        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')

    def compute(self, inputs, outputs):
        """
        Evaluates the equation
        y1 = z1**2 + z2 + x1 - 0.2*y2
        """
        z1 = inputs['z1']
        z2 = inputs['z2']
        x1 = inputs['x']
        y2 = inputs['y2']

        outputs['y1'] = z1**2 + z2 + x1 - 0.2*y2

In [4]:
class SellarDis2(om.ExplicitComponent):
    """
    Component containing Discipline 2 -- no derivatives version.
    """
    
    def initialize(self):
        self.options.declare('num_nodes', types=int, default=100)

    def setup(self):
        nn = self.options["num_nodes"]
        
        # Global Design Variable
        self.add_input('z1', val=np.zeros(nn,))
        self.add_input('z2', val=np.zeros(nn,))

        # Coupling parameter
        self.add_input('y1', val=np.ones(nn,))

        # Coupling output
        self.add_output('y2', val=np.ones(nn,))

    def setup_partials(self):
        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')

    def compute(self, inputs, outputs):
        """
        Evaluates the equation
        y2 = y1**(.5) + z1 + z2
        """

        z1 = inputs['z1']
        z2 = inputs['z2']
        y1 = inputs['y1']

        # Note: this may cause some issues. However, y1 is constrained to be
        # above 3.16, so lets just let it converge, and the optimizer will
        # throw it out
        # if y1.real < 0.0:
        #     y1 *= -1
        mask = y1.real < 0.0
        y1[mask] = -y1[mask]

        outputs['y2'] = y1**.5 + z1 + z2

#### Connections

In [5]:
class SellarMDA(om.Group):
    """
    Group containing the Sellar MDA.
    """
    
    def initialize(self):
        self.options.declare('num_nodes', types=int, default=100)

    def setup(self):
        nn = self.options["num_nodes"]
        cycle = self.add_subsystem('cycle', om.Group(), promotes=['*'])
        cycle.add_subsystem('d1', SellarDis1(num_nodes=nn), promotes_inputs=['x', 'z1', 'z2', 'y2'],
                            promotes_outputs=['y1'])
        cycle.add_subsystem('d2', SellarDis2(num_nodes=nn), promotes_inputs=['z1', 'z2', 'y1'],
                            promotes_outputs=['y2'])

        cycle.set_input_defaults('x', val=np.ones(nn,))
        cycle.set_input_defaults('z1', val=np.ones(nn,) * 5.0)
        cycle.set_input_defaults('z2', val=np.ones(nn,) * 2.0)

        # Nonlinear Block Gauss Seidel is a gradient free solver
        cycle.nonlinear_solver = om.NonlinearBlockGS()

        self.add_subsystem(
            'obj_cmp', 
            om.ExecComp(
                'obj = x**2 + z1 + y1 + exp(-y2)',
                x=np.zeros(nn,),
                z1=np.zeros(nn,), 
                y1=np.zeros(nn,),
                y2=np.zeros(nn,),
                obj=np.zeros(nn,),
            ),
            promotes=['x', 'z1', 'y1', 'y2', 'obj'])
        self.add_subsystem(
            'con_cmp1', 
            om.ExecComp(
                'con1 = 3.16 - y1',
                y1=np.zeros(nn,),
                con1=np.zeros(nn,),
            ), 
            promotes=['con1', 'y1']
        )
        self.add_subsystem(
            'con_cmp2', 
            om.ExecComp(
                'con2 = y2 - 24.0',
                y2=np.zeros(nn,),
                con2=np.zeros(nn,),
            ), 
            promotes=['con2', 'y2']
        )

In [6]:
prob = om.Problem(model=SellarMDA(num_nodes=1)).setup()  # note: vectorized optimization not possible with openmdao (num_nodes=1)

Visualize connections: 

In [7]:
om.n2(prob)

#### Optimization

Use MDAO to solve the optimization problem: 

In [8]:
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
# prob.driver.options['maxiter'] = 100
prob.driver.options['tol'] = 1e-8

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z1', lower=0, upper=10)
prob.model.add_design_var('z2', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()

prob.setup()
prob.set_solver_print(level=0)
prob.run_driver()

print('minimum found at')
print(prob.get_val('x')[0])
print(prob.get_val('z1'))
print(prob.get_val('z2'))

print('minumum objective')
print(prob.get_val('obj')[0])

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.162086954524598
            Iterations: 11
            Function evaluations: 11
            Gradient evaluations: 11
Optimization Complete
-----------------------------------
minimum found at
5.908901840045999e-16
[7.85470778e-15]
[4.39440972]
minumum objective
3.162086954524598


### Model Visualization

We setup a separate `Group` that does not include a solver, which we will use to understand local sensitivities in the vicininity of the optimum. 

In [9]:
class SellarView(om.Group):
    
    def initialize(self):
        self.options.declare('num_nodes', types=int, default=100)

    def setup(self):
        nn = self.options["num_nodes"]
        self.add_subsystem('d1', SellarDis1(num_nodes=nn), promotes_inputs=['x', 'z1', 'z2', 'y2'],
                            promotes_outputs=[('y1', 'y1_out')])
        self.add_subsystem('d2', SellarDis2(num_nodes=nn), promotes_inputs=['z1', 'z2', 'y1'],
                            promotes_outputs=[('y2', 'y2_out')])

        self.set_input_defaults('x', val=np.ones(nn,))
        self.set_input_defaults('y1', val=np.ones(nn,))
        self.set_input_defaults('y2', val=np.ones(nn,))
        self.set_input_defaults('z1', val=np.ones(nn,) * 5.0)
        self.set_input_defaults('z2', val=np.ones(nn,) * 2.0)
        
        self.add_subsystem(
            'res1', 
            om.ExecComp(
                'R1 = y1_out - y1',
                R1=np.zeros(nn,),
                y1_out=np.zeros(nn,),
                y1=np.zeros(nn,),
            ), 
            promotes=['R1', 'y1_out', "y1"]
        )
        self.add_subsystem(
            'res2', 
            om.ExecComp(
                'R2 = y2_out - y2',
                R2=np.zeros(nn,),
                y2_out=np.zeros(nn,),
                y2=np.zeros(nn,),
            ), 
            promotes=['R2', 'y2_out', "y2"]
        )
        self.add_subsystem(
            'obj_cmp', 
            om.ExecComp(
                'obj = x**2 + z1 + y1 + exp(-y2)',
                x=np.zeros(nn,),
                z1=np.zeros(nn,), 
                y1=np.zeros(nn,),
                y2=np.zeros(nn,),
                obj=np.zeros(nn,),
            ),
            promotes=['x', 'z1', 'y1', 'y2', 'obj'])
        self.add_subsystem(
            'con_cmp1', 
            om.ExecComp(
                'con1 = 3.16 - y1',
                y1=np.zeros(nn,),
                con1=np.zeros(nn,),
            ), 
            promotes=['con1', 'y1']
        )
        self.add_subsystem(
            'con_cmp2', 
            om.ExecComp(
                'con2 = y2 - 24.0',
                y2=np.zeros(nn,),
                con2=np.zeros(nn,),
            ), 
            promotes=['con2', 'y2']
        )

In [18]:
profiler = ipymodelview.openmdao_profiler(
    problem=om.Problem(model=SellarView()).setup(),
    inputs=[
        ("x", 0, 10, None), 
        ("z1", 0, 3, None), 
        ("z2", 0, 10, None), 
        ("y1", 0, 10, None), 
        ("y2", 0, 10, None), 
    ],
    outputs=[
        ("R1", -5, 5, None), 
        ("R2", -5, 5, None), 
        ("obj", -5, 50, None), 
        ("con1", -5, 5, None), 
        ("con2", -50, 0, None), 
    ],
    defaults=dict(
        x=dict(val=prob.get_val("x"), units=None),
        z1=dict(val=prob.get_val("z1"), units=None),
        z2=dict(val=prob.get_val("z2"), units=None),
        y1=dict(val=prob.get_val("y1"), units=None),
        y2=dict(val=prob.get_val("y2"), units=None),
    ),
    resolution=100,
    width=200,
    height=None,
)
profiler.view

View(children=(GridspecLayout(children=(Figure(axes=[Axis(label='x', num_ticks=3, scale=LinearScale(max=10.0, …

In [19]:
profiler.controller

Controller(children=(FloatSlider(value=5.908901840045999e-16, description='x', max=10.0), FloatSlider(value=7.…